# FIN 514 - Crank-Nicolson Finite Difference - Barrier options
**Spring 2021**

This notebook provides two differenyt Crank-Nicolson Finite Difference Code to value Barrier style options. We'll use the case in PS3 as the benchmark

## Packages and Configurations

The following common packages will be use on this notebook.

* numpy - [https://numpy.org/](https://numpy.org/)
* Pandas - [https://pandas.pydata.org/](https://pandas.pydata.org/)
* matplotlib - [https://matplotlib.org/](https://matplotlib.org/)
* Scipy Statistical functions - [https://docs.scipy.org/doc/scipy/reference/stats.html](https://docs.scipy.org/doc/scipy/reference/stats.html)


In [1]:
import numpy as np
import pandas as pd
import scipy.stats as st
import matplotlib.pyplot as plt

In [17]:
# ENTER INPUT FOR: start_step

Face = 1000
jmax = 100 #Number of S steps
imax = 5000 #Number of t steps
S0 =  3719.04
K = 2789.28
Barr = 2789.28
sigma = 0.3
r1 = 0.1
r2 = 0.1
q = 0
Tm = 401/365
Tv = 398/365
tcp = [36/365,66/365,97/365,128/365,156/365,187/365,217/365,248/365,
278/365,309/365,340/365,370/365,401/365]
tob = [34/365,64/365,95/365,126/365,154/365,182/365,215/365,246/365,
276/365,307/365,338/365,368/365,398/365]
cpn = .116
imax = 5000
jmaxmin = 200
jmaxmax =500
jmaxstep = 50
tauto = 182/365

SL = 0 #Minimum S value
SU = 2.5*S0 #Maximum S value

## Crank-Nicolson Finite Difference Function - continuous

## Discrete barrier option with correct lambda

In [22]:
CNFD_discbarrier(Face, S0, K, Barr, Tv, Tm, tcp, tob, tauto, cpn, r1, r2, q, sigma, SU, jmaxmin, jmaxmax, jmaxstep, imax)

[{'S_steps': 200,
  't_steps': 5000,
  'CN': 908.7637282308074,
  'Barrier Lambda': 0.9999999999999963},
 {'S_steps': 250,
  't_steps': 5000,
  'CN': 908.8437387802438,
  'Barrier Lambda': 0.9999999999999987},
 {'S_steps': 300,
  't_steps': 5000,
  'CN': 908.8982381288255,
  'Barrier Lambda': 0.9999999999999915},
 {'S_steps': 350,
  't_steps': 5000,
  'CN': 908.9377332969285,
  'Barrier Lambda': 1.0000000000000036},
 {'S_steps': 400,
  't_steps': 5000,
  'CN': 908.9676645658224,
  'Barrier Lambda': 0.9999999999999866},
 {'S_steps': 450,
  't_steps': 5000,
  'CN': 908.9911279891973,
  'Barrier Lambda': 1.000000000000006}]

In [21]:

def CNFD_discbarrier(Face, S0, K, Barr, Tv, Tm, tcp, tob, tauto, cpn, r1, r2, q, sigma, SU, jmaxmin, jmaxmax, jmaxstep, imax):
    
    #LIST TO SAVE RESULTS
    #Assumes that SL= 0, can be relaxed, see notes for updates A, B, C values in this case
    
    cnfddb_result = []
    
    for jmax in range(jmaxmin, jmaxmax, jmaxstep): 
    
    # CREATE TWO DIMENSIONAL ARRAY OF SIZE [imax+1,jmax+1] TO STORE V AT ALL STEPS
    # V[imax, jmax]
    
        V1 = np.zeros([imax+1, jmax+1])
        V2 = np.zeros([imax+1, jmax+1])

    # CREATE ONE DIMENSIONAL ARRAY OF SIZE [jmax+1] TO STORE A, B, C VALUES AT ALL STEPS
    # A[jmax], B[jmax], C[jmax] 
    
        A = np.zeros([jmax+1])
        B = np.zeros([jmax+1])
        C = np.zeros([jmax+1])
        D = np.zeros([jmax+1])
        alpha = np.zeros([jmax+1])
        CN_S = np.zeros([jmax+1])
    
    # Set up time and S steps
    
        dt = Tv / imax
        dS = SU/jmax
        
        jb = int(Barr/dS)
        if (jb*dS > Barr): jb = jb-1 
        
        ji = int(S0/dS)
        if (ji*dS > S0): ji = ji-1
            
        icp1  = [j/dt for j in tcp] # coupon payment dates
        icp   = [int(j) for j in icp1] # nearest time point to coupon payment dates
        inc   = [j for j in icp if j<tauto]
        
        iob1  = [j/dt for j in tob] # observation date
        iob   = [int(j) for j in iob1] # nearest time point to observation date
        
        iauto    = [j for j in iob if j>=tauto ] # nearest time points to autocallable observation dates 
        ibarrier = iob
      
        
    # CALCULATE OPTION VALUES AND PROBABILITIES
    # Start at the last step number because we are going to be moving backwards from times step imax to times step 0
    
        i = imax 
        
        for j in range(0, jmax+1):    
    # Then, calculate the value of the option at that exact position within the binomial tree
    # Also calculate the probabilities A, B, C
           
            if j>ji:
                V1[i, j] = Face*(1+cpn)*np.exp(-r2*(Tm-Tv))
            elif j>jb: 
                V1[i, j] = (dS*j/S0)*Face*(1+cpn)*np.exp(-r2*(Tm-Tv))
            else: V1[i, j] = (dS*j/S0)*Face*np.exp(-r2*(Tm-Tv))

    #Now go back in time
        for i in range(imax-1, -1, -1):

            if (i in ibarrier):    
                #Lower boundary condition in matrix terms
                A[0] = 0
                B[0] = 1
                C[0] = 0
                D[0] = 0
    
                #regular D values
                for j in range(1, jmax, 1):
                    A[j] = 0.25*sigma**2*j**2 - 0.25*(r1-q)*j
                    B[j] = -1/dt-0.5*r1-0.5*sigma**2*j**2
                    C[j] = 0.25*sigma**2*j**2 + 0.25*(r1-q)*j
                    D1 = -V1[i+1,j-1]*(0.25*sigma**2*j**2-0.25*(r1-q)*j)
                    D2 = -V1[i+1,j]*(1/dt-0.5*r1-0.5*sigma**2*j**2)
                    D3 = -V1[i+1,j+1]*(0.25*sigma**2*j**2+0.25*(r1-q)*j)
                    D[j] = D1+D2+D3
            
                #Upper boundary condition in matrix terms
                A[jmax] = 0
                B[jmax] = 1
                C[jmax] = 0
                
                TNCi = tcp[iob.index(i)+1]
                
                if (i in iauto):
                    D[jmax] = Face*(1+cpn)*np.exp(-r2*(TNCi-dt*i))
                if (i in inc):
                    cpk = np.zeros(5)
                    for k in range(5):
                        if inc[k] >i:
                            cpk[k] = Face*cpn*np.exp(-r2(inc[k]*dt-i*dt)) # present value of coupons 

                    D[jmax] = Face*(1+cpn)*np.exp(-r2*(tauto-dt*i))*(sum(cpk))
            
                alpha[0] = B[0]
                CN_S[0] = D[0]
                for j in range(1, jmax+1, 1):
                    alpha[j] = B[j]-(A[j]*C[j-1])/alpha[j-1]
                    CN_S[j] = D[j]-(A[j]*CN_S[j-1])/alpha[j-1]  
                V1[i,jmax] = CN_S[jmax]/alpha[jmax]
                for j in range(jmax-1,-1,-1):
                    V1[i,j] = (CN_S[j]-C[j]*V1[i,j+1])/alpha[j]
            
            else:
            
                #Lower boundary condition in matrix terms
                A[0] = 0
                B[0] = 1
                C[0] = 0
                D[0] = 0

                #regular D values
                for j in range(1, jmax, 1):
                    A[j] = 0.25*sigma**2*j**2 - 0.25*(r1-q)*j
                    B[j] = -1/dt-0.5*r1-0.5*sigma**2*j**2
                    C[j] = 0.25*sigma**2*j**2 + 0.25*(r1-q)*j
                    D1 = -V1[i+1,j-1]*(0.25*sigma**2*j**2-0.25*(r1-q)*j)
                    D2 = -V1[i+1,j]*(1/dt-0.5*r1-0.5*sigma**2*j**2)
                    D3 = -V1[i+1,j+1]*(0.25*sigma**2*j**2+0.25*(r1-q)*j)
                    D[j] = D1+D2+D3

                #Upper boundary condition in matrix terms
                A[jmax] = 0
                B[jmax] = 1
                C[jmax] = 0
                D[jmax] = Face*(1+cpn)


                alpha[0] = B[0]
                CN_S[0] = D[0]
                for j in range(1, jmax+1, 1):
                    alpha[j] = B[j]-(A[j]*C[j-1])/alpha[j-1]
                    CN_S[j] = D[j]-(A[j]*CN_S[j-1])/alpha[j-1]  
                V1[i,jmax] = CN_S[jmax]/alpha[jmax]
                for j in range(jmax-1,-1,-1):
                    V1[i,j] = (CN_S[j]-C[j]*V1[i,j+1])/alpha[j]
                
    # RELAY OUTPUTS TO DICTIONARY
        jcritreal = S0/dS
        jcrit = int(jcritreal)
        jcritB = int(Barr/dS)+1
        Vcrit = V1[0,jcrit]+ (S0 - jcrit * dS) / (dS) * (V1[0,jcrit+1] - V1[0,jcrit])
        Blambda = (jcritB*dS - Barr)/dS
        output = {'S_steps': jmax, 't_steps': imax, 'CN': Vcrit, 'Barrier Lambda': Blambda}
        cnfddb_result.append(output)

    return cnfddb_result
